In [1]:
token = "" # here should be lichess API token, but i will hide my token because its confidential data

In [11]:
import pandas as pd
import numpy as np
import berserk

In [3]:
session = berserk.TokenSession(token)
client = berserk.Client(session=session)

In [286]:
duno = client.games.export_by_player('EvilSaintPL') # posible adding max=300, because without limit it takes some time

In [317]:
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(games, f, ensure_ascii=False, default=str) # saving data without any changes
some_chess_data = pd.read_json('data.json')
some_chess_data[:1] 

# im not gonna work on it, instead i will handle data without json.dump() to train working on dicts and list

id  rated   variant  speed   perf                         createdAt  \
0  myb1R9l1   True  standard  blitz  blitz  2023-05-31 14:06:14.961000+00:00   

                         lastMoveAt  status  \
0  2023-05-31 14:14:46.534000+00:00  resign   

                                             players winner  \
0  {'white': {'user': {'name': 'sadasukh', 'id': ...  black   

                                               moves  \
0  e4 e5 d3 d5 exd5 Qxd5 c4 Qc6 Nc3 Nf6 Bg5 Be7 B...   

                                               clock tournament initialFen  \
0  {'initial': 300, 'increment': 0, 'totalTime': ...        NaN        NaN   

  swiss  daysPerTurn  
0   NaN          NaN

In [247]:
games = list(duno)

In [318]:
games[:2]
# at this point im looing into data, and deciding what data do i need"
# in this case i dont need moves, only a amount of moves played by side
# also its not important if was a game played in tourament
# days per turn have value only for corespondic chess games so i can drop it too
# swiss and initialFen also are not important
# from players -> user i need only one result, so i will take name
# ratingDiff will be calculated later on my own

[{'id': 'myb1R9l1',
  'rated': True,
  'variant': 'standard',
  'speed': 'blitz',
  'perf': 'blitz',
  'createdAt': datetime.datetime(2023, 5, 31, 14, 6, 14, 961000, tzinfo=datetime.timezone.utc),
  'lastMoveAt': datetime.datetime(2023, 5, 31, 14, 14, 46, 534000, tzinfo=datetime.timezone.utc),
  'status': 'resign',
  'players': {'white': {'user': {'name': 'sadasukh', 'id': 'sadasukh'},
    'rating': 1728,
    'ratingDiff': -5},
   'black': {'user': {'name': 'EvilSaintPL', 'id': 'evilsaintpl'},
    'rating': 1768,
    'ratingDiff': 5}},
  'winner': 'black',
  'moves': 'e4 e5 d3 d5 exd5 Qxd5 c4 Qc6 Nc3 Nf6 Bg5 Be7 Bxf6 Bxf6 Nd5 O-O Nf3 Be6 Rc1 Bxd5 cxd5 Qxd5 Be2 c6 b4 Nd7 O-O Qxa2 Nd2 Qb2 Rb1 Qd4 Ne4 Be7 b5 cxb5 Rxb5 b6 Bg4 Nc5 Nxc5 Bxc5 Qf3 Rad8 Bf5 Qf4 Qxf4 exf4 Be4 f5 Bb7 Rxd3 Rbb1 Rb8 Ba6 Ra3 Bc4+ Kf8 Rbe1 Re8 Rd1 Rc3 Bb5 Re7 Rd8+ Kf7 h4 Rc2 Rd5 g6 Ba4 Ra2 Bb3 Ra3 Bc4 Kf6 h5 a5 hxg6 hxg6 Bb5 Rb3 Bc4 Rb2 Rdd1 a4 Rb1 a3 Rxb2 axb2 Rb1 Ba3 f3 Rc7 Ba2 Rc1+ Kh2 Rxb1 Bxb1 Ke5 g3 fxg3+ Kxg3 

# Changing to valueable data   - NEED TO ADD NUMBER OF MOVES IN GAMES

In [270]:
cols= ['id', 'rated', 'variant', 'speed', 'createdAt', 'lastmoveAt', 'status',
       'winner', 'clock', 'increment', 'white_player', 'white_rating',
       'black_player', 'black_rating', 'ratingDiff']
chessdata = pd.DataFrame(columns=cols)

for i in range(len(games)):
    if i%100==0:
        print(i)
    df = pd.DataFrame.from_dict(games[i])
    if (df.loc["white","speed"]!="correspondence") & ('user' in df.loc["black","players"].keys()) & ('user' in df.loc["white","players"].keys()):
        if (df.loc["white", "status"]!="draw") & (df.loc["white", "status"]!="stalemate") & (df.loc["white", "status"]!="outoftime") & (df.loc["white", "status"]!="timeout"):
            d = {
            "id" : df.loc["white", "id"],
            "rated": df.loc["white", "rated"],
            "variant": df.loc["white", "variant"],
            "speed": df.loc["white", "speed"],
            "createdAt": df.loc["white", "createdAt"],
            "lastmoveAt": df.loc["white", "lastMoveAt"],
            "status": df.loc["white", "status"],
            "winner": df.loc["white", "winner"],
            "clock": df.loc["initial","clock"],
            "increment": df.loc["increment","clock"],
            "white_player": df.loc["white","players" ]["user"]["name"],
            "white_rating": df.loc["white","players" ]['rating'],
            "black_player": df.loc["black","players" ]["user"]["name"],
            "black_rating": df.loc["black","players" ]['rating'],
            }
            d = pd.Series(d)
            d = pd.DataFrame(data=d)
            d = d.transpose()
            chessdata = pd.concat([chessdata, d])
        else:
            d = {
            "id" : df.loc["white", "id"],
            "rated": df.loc["white", "rated"],
            "variant": df.loc["white", "variant"],
            "speed": df.loc["white", "speed"],
            "createdAt": df.loc["white", "createdAt"],
            "lastmoveAt": df.loc["white", "lastMoveAt"],
            "status": df.loc["white", "status"],
            "winner": 0,
            "clock": df.loc["initial","clock"],
            "increment": df.loc["increment","clock"],
            "white_player": df.loc["white","players" ]["user"]["name"],
            "white_rating": df.loc["white","players" ]['rating'],
            "black_player": df.loc["black","players" ]["user"]["name"],
            "black_rating": df.loc["black","players" ]['rating'],
            }
            d = pd.Series(d)
            d = pd.DataFrame(data=d)
            d = d.transpose()
            chessdata = pd.concat([chessdata, d])
    

a0
a1
a2
a3
a4
a5
a6
a7
b8
a9
a10
a11
b12
a13
a14
a15
b16
a17
b18
a19
b20
a21
b22
a23
a24
b25
b26
b27
b28
b29
b30
a31
a32
a33
a34
a35
a36
a37
b38
b39
a40
a41
b42
a43
a44
b45
b46
a47
a48
a49
a50
b51
b52
a53
a54
a55
a56
a57
a58
a59
a60
a61
a62
a63
a64
a65
a66
a67
a68
a69
b70
a71
a72
a73
a74
a75
a76
a77
b78
a79
b80
a81
a82
a83
a84
b85
a86
a87
a88
b89
a90
b91
a92
a93
b94
a95
a96
a97
b98
a99
b100
b101
a102
a103
a104
a105
b106
a107
a108
a109
a110
a111
b112
a113
a114
a115
a116
b117
a119
a120
a121
a122
a123
a124
a125
a126
a127
a128
a129
a130
a131
a132
a133
a134
b135
a136
a137
a138
b139
b140
a141
b142
a143
a144
a145
b146
a147
a148
a149
a150
a151
a152
b153
b154
b155
a156
a157
a158
a159
a160
a161
a162
b163
a164
a165
a166
b167
a168
a169
a170
b171
a172
b173
a174
a175
b176
a177
a178
a179
a180
b181
a182
a183
b184
a185
a186
b187
a188
a189
a190
a191
a192
a193
a194
b195
a196
a197
a198
a199
a200
a201
b202
a203
a204
b205
a206
a207
a208
a209
a210
a211
a212
a213
b214
a215
b216
a217
a218
a219
a220
a221
b222


a1584
a1585
a1586
a1587
b1588
b1589
a1590
a1591
a1592
a1593
a1594
a1595
a1596
a1597
a1598
a1599
a1600
a1601
a1602
a1603
a1604
a1605
a1606
a1607
a1608
a1609
a1610
a1611
b1612
a1613
a1614
a1615
a1616
a1617
b1618
a1619
a1620
b1621
b1622
b1623
b1624
b1625
a1626
a1627
a1628
a1629
a1630
a1631
b1632
a1633
a1634
a1635
a1636
a1637
a1638
a1639
b1640
b1641
a1642
b1643
b1644
a1645
a1646
b1647
b1648
b1649
b1650
a1651
a1652
a1653
a1654
b1655
a1656
a1657
a1658
b1659
a1660
a1661
b1662
b1663
b1664
b1665
b1666
a1667
b1668
b1669
a1670
b1671
a1672
b1673
a1674
a1675
a1676
a1677
a1678
a1679
a1680
a1681
a1682
a1683
b1684
b1685
b1686
a1687
a1688
a1689
a1690
a1691
a1692
b1693
a1694
a1695
a1696
a1697
a1698
a1699
a1700
a1701
a1702
a1703
a1704
b1705
a1706
b1707
a1708
b1709
b1711
a1712
a1713
a1714
b1715
a1716
a1717
b1718
a1719
a1720
a1721
a1722
a1723
a1724
b1725
b1726
a1727
a1728
a1729
a1730
a1731
a1732
a1733
a1734
a1735
a1736
a1737
a1738
b1739
a1740
a1741
a1742
a1743
a1744
a1745
a1746
a1747
b1748
a1749
b1750
a175

a3007
b3008
a3009
a3010
a3011
b3012
a3013
a3014
a3015
a3016
a3017
a3018
a3019
a3020
a3022
a3023
b3024
a3025
a3026
a3027
a3028
a3029
b3030
a3031
a3032
b3033
a3034
a3035
b3036
b3037
a3038
b3039
b3040
a3041
a3042
a3043
a3044
a3045
a3046
a3047
a3048
a3049
a3050
a3051
b3052
a3053
a3054
a3055
a3056
b3057
a3058
a3059
a3060
a3061
a3062
a3063
a3064
b3065
a3066
a3067
b3068
a3069
a3070
a3071
a3072
a3073
a3074
a3075
a3076
a3077
b3078
a3079
a3080
a3081
a3082
a3083
a3084
a3085
a3086
a3087
a3088
a3089
a3090
a3091
b3092
a3093
a3094
b3095
a3096
a3097
a3098
a3099
a3100
b3101
a3102
a3103
a3104
a3105
b3106
a3107
a3108
a3109
a3110
a3111
a3112
a3113
b3114
a3115
a3116
a3117
a3118
a3119
a3120
a3121
a3122
b3123
a3124
a3125
b3126
b3127
a3128
a3129
a3130
b3131
b3132
a3133
a3134
a3135
a3136
b3137
a3138
a3139
a3140
a3141
a3142
a3143
a3144
b3145
b3146
a3147
a3148
a3149
a3150
a3151
a3152
b3153
a3154
a3155
a3156
b3157
a3158
b3159
a3160
a3161
a3162
a3163
a3164
a3165
a3166
a3167
a3168
a3169
a3170
b3171
b3172
a3173
a317

a4479
b4480
a4481
a4482
a4483
a4484
a4485
a4486
a4487
b4488
b4489
a4490
a4491
a4492
a4493
a4494
a4496
a4497
b4498
a4499
a4500
b4501
b4502
a4503
b4504
b4505
a4506
a4507
a4508
a4509
a4510
a4511
a4512
a4513
a4514
a4515
b4516
b4517
a4518
a4519
b4520
a4521
a4522
b4523
a4524
a4525
a4526
a4527
a4528
a4529
a4530
b4531
a4532
a4533
a4534
a4535
a4536
a4537
a4538
a4539
a4540
b4541
a4542
b4543
a4544
a4545
a4546
b4547
a4548
a4549
a4550
a4551
a4552
a4553
a4554
a4555
a4556
b4557
a4558
b4559
a4560
a4561
a4562
b4563
b4564
b4565
b4566
a4567
a4568
a4569
a4570
b4571
a4572
a4573
a4574
b4575
a4576
a4577
a4578
a4579
b4580
b4581
a4582
a4583
a4584
a4585
a4586
a4587
a4588
a4589
a4590
a4591
a4592
b4593
a4594
a4595
a4596
a4597
a4598
b4599
b4600
a4601
b4602
a4603
a4604
a4605
b4606
a4607
a4608
a4609
a4610
a4611
a4612
a4613
b4614
a4615
b4616
a4617
a4618
a4619
a4620
b4621
b4622
a4623
a4624
a4625
a4626
a4627
a4628
b4629
a4630
a4631
b4632
a4633
a4634
b4635
a4636
a4637
b4638
a4639
b4640
a4641
a4642
a4643
b4644
a4645
b464

In [271]:
chessdata=chessdata.reset_index()
chessdata.drop("index",axis=1,inplace=True)

In [272]:
chessdata.shape

(5461, 15)

In [273]:
chessdata

id rated   variant  speed                        createdAt  \
0     myb1R9l1  True  standard  blitz 2023-05-31 14:06:14.961000+00:00   
1     6wWH5djx  True  standard  blitz 2023-05-31 14:02:18.498000+00:00   
2     dchTadkN  True  standard  blitz 2023-05-31 14:00:26.666000+00:00   
3     hkG3xcPk  True  standard  blitz 2023-05-31 13:57:58.420000+00:00   
4     yN0XtQ2l  True  standard  blitz 2023-05-31 13:48:37.354000+00:00   
...        ...   ...       ...    ...                              ...   
5456  dYnYtNFc  True  standard  blitz 2019-11-09 19:53:37.672000+00:00   
5457  HLTrX7cv  True  standard  blitz 2019-11-09 19:51:04.051000+00:00   
5458  enP16ebg  True  standard  blitz 2019-11-09 19:46:10.220000+00:00   
5459  8cRwRfdY  True  standard  blitz 2019-11-09 19:45:24.715000+00:00   
5460  3D9bHutq  True  standard  blitz 2019-11-09 19:42:13.341000+00:00   

                           lastmoveAt     status winner  clock increment  \
0    2023-05-31 14:14:46.534000+00:00     resign  black  300.0       0.0   
1    2023-05-31 14:06:08.978000+00:00     resign  white  300.0       0.0   
2    2023-05-31 14:02:11.921000+00:00     resign  white  300.0       0.0   
3    2023-05-31 13:58:52.940000+00:00       mate  black  300.0       0.0   
4    2023-05-31 13:57:41.046000+00:00     resign  white  300.0       0.0   
...                               ...        ...    ...    ...       ...   
5456 2019-11-09 19:59:37.804000+00:00  outoftime      0  180.0       0.0   
5457 2019-11-09 19:53:25.908000+00:00       mate  white  180.0       0.0   
5458 2019-11-09 19:49:53.831000+00:00       mate  black  180.0       0.0   
5459 2019-11-09 19:46:02.537000+00:00     resign  white  180.0       0.0   
5460 2019-11-09 19:45:09.538000+00:00       mate  black  180.0       0.0   

       white_player white_rating black_player black_rating ratingDiff  
0          sadasukh         1728  EvilSaintPL         1768        NaN  
1       EvilSaintPL         1763     sadasukh         1733        NaN  
2         andrejpan         1731  EvilSaintPL         1769        NaN  
3           Minik55         1801  EvilSaintPL         1763        NaN  
4       EvilSaintPL         1757   germanhugo         1749        NaN  
...             ...          ...          ...          ...        ...  
5456    EvilSaintPL         1206      zfred12         1311        NaN  
5457        zfred12         1305  EvilSaintPL         1272        NaN  
5458     Mokrakifla         1061  EvilSaintPL         1216        NaN  
5459  NecroCosmos98         1347  EvilSaintPL         1326        NaN  
5460    EvilSaintPL         1500      Waddle8         1503        NaN  

[5461 rows x 15 columns]